# Results
> code for plotting results and comparing with MDS and ERA gap filling

In [ ]:
#| hide
#|default_exp meteo_imp.results

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from fastcore.test import *
from fastcore.basics import *
from meteo_imp.utils import *
from meteo_imp.gaussian import *
from meteo_imp.data_preparation import MeteoDataTest
from meteo_imp.kalman.filter import *
from meteo_imp.kalman.filter import get_test_data
from meteo_imp.data import *
from meteo_imp.kalman.fastai import *
from fastcore.transform import *
from fastai.learner import *
from pyprojroot import here

import pykalman
from typing import *

import numpy as np
import pandas as pd
import torch
from torch import Tensor
from torch.distributions import MultivariateNormal

from timeit import timeit
import polars as pl
import altair as alt

from tqdm.auto import tqdm

import io
from contextlib import redirect_stderr

load data for testing

In [ ]:
reset_seed()

In [ ]:
hai = pd.read_parquet(hai_big_path)
hai_era = pd.read_parquet(hai_era_big_path)

In [ ]:
dls = imp_dataloader(hai, hai_era, var_sel = 'TA', gap_len=5, block_len=100, control_lags = [1], n_rep=1, bs=1).cpu()

In [ ]:
item = MeteoImpItem(1,2, 'TA', gap_len=10)
items = [item]

In [ ]:
targ = orig_target(dls, items)[0]

In [ ]:
input, _ = one_batch_with_items(dls, items)

## Kalman Filter

take a `MeteoImp Df` and gap fill it using the given model

In [ ]:
model = torch.load(here("analysis/results/trained_3_feb_TA_gap_5_v3.pickle"))

In [ ]:
preds, targs = predict_items(model, dls=dls, items = items)
preds[0].mean[targs[0].mask] = targs[0].data[targs[0].mask]

### KalmanImputation

In [ ]:
class KalmanImputation:
    name = "KalmanFilter"
    def __init__(self, model): store_attr()
    def __call__(self, item, dls):
        pred, targ = predict_items(model, dls=dls, items = [item])
        pred[0].mean[targ[0].mask] = targ[0].data[targ[0].mask]
        return pred[0].mean
    def preds_all(self, items, dls):
        return predict_items(model, dls=dls, items = items)

In [ ]:
k_imp = KalmanImputation(model)

In [ ]:
pred = k_imp(item, dls=dls)

In [ ]:
display_as_row({'pred': pred[45:55],'data': targ.data[45:55], 'mask': targ.mask[45:55]}, hide_idx=False)

,TA,SW_IN,VPD,PA,P,WS,LW_IN
time,,,,,,,
2000-01-04 02:30:00,2.1900,0.0000,0.9520,96.3000,0.0000,5.3900,304.7310
2000-01-04 03:00:00,2.2700,0.0000,0.9710,96.3000,0.0000,5.5900,304.7310
2000-01-04 03:30:00,2.3200,0.0000,0.9580,96.2900,0.0000,5.6700,304.7310
2000-01-04 04:00:00,2.3400,0.0000,0.9380,96.2600,0.0000,6.6100,304.7310
2000-01-04 04:30:00,2.2400,0.0000,0.7850,96.2600,0.0000,6.6500,315.0050
2000-01-04 05:00:00,2.1409,0.0000,0.5290,96.2700,0.0000,6.1200,315.0050
2000-01-04 05:30:00,2.0485,0.0000,0.4500,96.2600,0.2900,5.5800,315.0050
2000-01-04 06:00:00,2.0708,0.0000,0.4990,96.2500,0.0000,4.7000,315.0050
2000-01-04 06:30:00,2.1016,0.0000,0.3820,96.2300,0.0000,5.0700,315.0050


## MDS

Need to call R from python

In [ ]:
#| export
import rpy2.robjects
from rpy2.robjects import r
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [ ]:
r.print('here::here("R/REddyProc_tools.R")')

[1] "here::here(\"R/REddyProc_tools.R\")"


"'here::here(""R/REddyProc_tools.R"")'"


In [ ]:
#| export
path_r = str(here("R/REddyProc_tools.R"))

r.source(path_r);

In [ ]:
#| export
def importr_install(pkg):
    try:
        importr(pkg)
    except:
        utils = importr('utils')
        utils.chooseCRANmirror(ind=1)
        utils.install_packages(pkg)
        importr(pkg) 

In [ ]:
#| export
def setupR():
    importr_install('tidyverse')
    importr_install('REddyProc')
    importr('lubridate')
    path_r = str(here("R/REddyProc_tools.R"))
    r.source(path_r) # R functions
    r("""toR_timestamp <- function(x){
   x$TIMESTAMP_END = as_datetime(x$TIMESTAMP_END) 
    x
     }""")

In [ ]:
#| export
def pd2R(x):
    with ro.default_converter + pandas2ri.converter:
        return ro.conversion.py2rpy(x) 

def R2pd(x):
    with ro.default_converter + pandas2ri.converter:
        return ro.conversion.rpy2py(x) 

In [ ]:
#| export
setupR() #start the R process and load depenencies

### Experiment

The time series needs to be at least 90 days long, we we add 45 days before and after the gap

There is a problem with conversions of timestamps between R and Python, so convert to string in Python and then back to datetime in R

In [ ]:
#| export
def add_buffer(index, inner_index, n):
    """Adds  a buffer of after and after index so length is at least """
    start = int(np.argwhere(index == inner_index[0]).squeeze())
    end = int(np.argwhere(index == inner_index[-1]).squeeze())
    start = start - n //2
    end = end + n//2
    if start < 0:
        end += -start
        start = 0
    if end > len(index):
        start -= end - len(index)
        end = len(index)
    
    index = index[start:end]
    
    assert len(index) > n
    
    return index

In [ ]:
add_buffer(hai.index, hai.index[:100], 50)
add_buffer(hai.index, hai.index[-50:], 50);

In [ ]:
#| export
def item2REddy(item, var, df):
    " Add context around item for supporting REddyProc"
    index = add_buffer(hai.index, item.data.index, 90 * 24 * 2)
    REddy_df = hai.loc[index].assign(gap = (~item.mask[var]).astype(int)).fillna({'gap': 0})
    return REddy_df.reset_index().astype({'time': str}).rename(columns={'time': 'TIMESTAMP_END'})

In [ ]:
REddy_df = item2REddy(targ, 'TA', hai)

REddy_df_r = r.toR_timestamp(pd2R(REddy_df))

filled = R2pd(r.fill_gaps_EProc(REddy_df_r, "TA"))

R[write to console]: New sEddyProc class for site 'ID'

R[write to console]: Initialized variable 'TA' with 10 real gaps for gap filling.

R[write to console]: Limited MDS algorithm for gap filling of 'TA.gap_0' with LUT(SW_IN only) and MDC.

R[write to console]: Look up table with window size of 7 days with SW_IN

R[write to console]: 10

R[write to console]: Finished gap filling of 'TA' in 0 seconds. Artificial gaps filled: 4419, real gaps filled: 10, unfilled (long) gaps: 0.



In [ ]:
filled

,TA_orig,TA_f,TA_fqc,TA_fall,TA_fall_qc,TA_fnum,TA_fsd,TA_fmeth,TA_fwin
1,-0.60,-0.60,0.0,-0.60,NaN,NaN,NaN,NaN,NaN
2,-0.65,-0.65,0.0,-0.65,NaN,NaN,NaN,NaN,NaN
3,-0.58,-0.58,0.0,-0.58,NaN,NaN,NaN,NaN,NaN
4,-0.51,-0.51,0.0,-0.51,NaN,NaN,NaN,NaN,NaN
5,-0.49,-0.49,0.0,-0.49,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4415,3.30,3.30,0.0,3.30,NaN,NaN,NaN,NaN,NaN
4416,3.10,3.10,0.0,3.10,NaN,NaN,NaN,NaN,NaN
4417,3.05,3.05,0.0,3.05,NaN,NaN,NaN,NaN,NaN
4418,3.05,3.05,0.0,3.05,NaN,NaN,NaN,NaN,NaN


In [ ]:
REddy_df.set_index("TIMESTAMP_END") 

,TA,SW_IN,VPD,PA,P,WS,LW_IN,gap
TIMESTAMP_END,,,,,,,,
2000-01-01 00:30:00,-0.60,0.0,0.222,96.63,0.0,2.05,302.475,0.0
2000-01-01 01:00:00,-0.65,0.0,0.122,96.58,0.0,2.53,302.475,0.0
2000-01-01 01:30:00,-0.58,0.0,0.090,96.56,0.0,3.15,301.677,0.0
2000-01-01 02:00:00,-0.51,0.0,0.110,96.56,0.0,3.12,301.677,0.0
2000-01-01 02:30:00,-0.49,0.0,0.102,96.57,0.0,3.04,301.677,0.0
...,...,...,...,...,...,...,...,...
2000-04-01 23:30:00,3.30,0.0,1.092,95.06,0.0,2.50,283.856,0.0
2000-04-02 00:00:00,3.10,0.0,1.013,95.04,0.0,2.09,283.856,0.0
2000-04-02 00:30:00,3.05,0.0,0.964,95.04,0.0,2.42,283.856,0.0


In [ ]:
filled = filled.set_index(pd.to_datetime(REddy_df.TIMESTAMP_END))

filled_item = filled.loc[targ.data.index]

pred = targ.data.copy()
pred.loc[~targ.mask["TA"], "TA"] = filled_item["TA_f"][~targ.mask["TA"]]

In [ ]:
pred[45:55], targ.data[45:55]

(                           TA  SW_IN    VPD     PA     P    WS    LW_IN
 time                                                                   
 2000-01-04 02:30:00  2.190000    0.0  0.952  96.30  0.00  5.39  304.731
 2000-01-04 03:00:00  2.270000    0.0  0.971  96.30  0.00  5.59  304.731
 2000-01-04 03:30:00  2.320000    0.0  0.958  96.29  0.00  5.67  304.731
 2000-01-04 04:00:00  2.340000    0.0  0.938  96.26  0.00  6.61  304.731
 2000-01-04 04:30:00  2.240000    0.0  0.785  96.26  0.00  6.65  315.005
 2000-01-04 05:00:00  1.786820    0.0  0.529  96.27  0.00  6.12  315.005
 2000-01-04 05:30:00  1.774891    0.0  0.450  96.26  0.29  5.58  315.005
 2000-01-04 06:00:00  1.763720    0.0  0.499  96.25  0.00  4.70  315.005
 2000-01-04 06:30:00  1.751759    0.0  0.382  96.23  0.00  5.07  315.005
 2000-01-04 07:00:00  1.741274    0.0  0.313  96.24  0.00  5.23  315.005,
                        TA  SW_IN    VPD     PA     P    WS    LW_IN
 time                                                 

In [ ]:
#| export
def gap_fill_item(item, REddy_df, var, filled):
    
    filled = filled.set_index(pd.to_datetime(REddy_df.TIMESTAMP_END))
    filled_item = filled.loc[item.data.index]

    pred = item.data.copy()
    pred.loc[~item.mask[var], var] = filled_item[f"{var}_f"][~item.mask[var]]
    return pred

In [ ]:
gap_fill_item(targ, REddy_df, "TA", filled)


,TA,SW_IN,VPD,PA,P,WS,LW_IN
time,,,,,,,
2000-01-03 04:00:00,0.81,0.0,0.000,97.00,0.0,3.53,304.148
2000-01-03 04:30:00,0.95,0.0,0.000,96.98,0.0,3.99,304.382
2000-01-03 05:00:00,1.09,0.0,0.000,96.96,0.0,3.61,304.382
2000-01-03 05:30:00,1.18,0.0,0.009,96.93,0.0,3.90,304.382
2000-01-03 06:00:00,1.35,0.0,0.061,96.91,0.0,4.17,304.382
...,...,...,...,...,...,...,...
2000-01-05 03:30:00,4.62,0.0,1.162,95.91,0.0,6.53,330.202
2000-01-05 04:00:00,4.51,0.0,1.636,95.94,0.0,5.76,330.202
2000-01-05 04:30:00,4.11,0.0,1.746,95.98,0.0,5.79,299.320


### MDSImputation

In [ ]:
#| export
class MDSImputation:
    name = "MDS"
    def __init__(self, var, df):
        store_attr()
        self.out = io.StringIO()
    def __call__(self, item):
        REddy_df = item2REddy(item, self.var, self.df)
        REddy_df_r = r.toR_timestamp(pd2R(REddy_df))
        with redirect_stderr(self.out):
            filled = R2pd(r.fill_gaps_EProc(REddy_df_r, self.var))
        return gap_fill_item(item, REddy_df, self.var, filled)

In [ ]:
mds_imp = MDSImputation('TA', hai)

In [ ]:
mds_imp(targ)

,TA,SW_IN,VPD,PA,P,WS,LW_IN
time,,,,,,,
2000-01-03 04:00:00,0.81,0.0,0.000,97.00,0.0,3.53,304.148
2000-01-03 04:30:00,0.95,0.0,0.000,96.98,0.0,3.99,304.382
2000-01-03 05:00:00,1.09,0.0,0.000,96.96,0.0,3.61,304.382
2000-01-03 05:30:00,1.18,0.0,0.009,96.93,0.0,3.90,304.382
2000-01-03 06:00:00,1.35,0.0,0.061,96.91,0.0,4.17,304.382
...,...,...,...,...,...,...,...
2000-01-05 03:30:00,4.62,0.0,1.162,95.91,0.0,6.53,330.202
2000-01-05 04:00:00,4.51,0.0,1.636,95.94,0.0,5.76,330.202
2000-01-05 04:30:00,4.11,0.0,1.746,95.98,0.0,5.79,299.320


## ERA Imputation

In [ ]:
class ERAImputation:
    name = "ERA"
    def __call__(self, item):
        pred = item.control.copy()
        names = [col for col in pred.columns if not col.endswith("_lag_1")]
        pred = pred.filter(names)
        return pred.rename(columns=lambda x: x.replace("_ERA", ""))

In [ ]:
era_imp = ERAImputation()

In [ ]:
era_imp(targ)

,TA,SW_IN,VPD,PA,P,WS,LW_IN
time,,,,,,,
2000-01-03 04:00:00,1.702,0.0,0.693,97.016,0.000,2.948,304.148
2000-01-03 04:30:00,1.762,0.0,0.691,97.002,0.026,2.989,304.382
2000-01-03 05:00:00,1.736,0.0,0.697,96.988,0.000,3.015,304.382
2000-01-03 05:30:00,1.711,0.0,0.703,96.974,0.000,3.041,304.382
2000-01-03 06:00:00,1.686,0.0,0.709,96.960,0.000,3.066,304.382
...,...,...,...,...,...,...,...
2000-01-05 03:30:00,4.945,0.0,0.774,95.910,0.252,6.107,330.202
2000-01-05 04:00:00,4.789,0.0,0.758,95.926,0.252,6.189,330.202
2000-01-05 04:30:00,4.632,0.0,0.741,95.942,0.044,6.270,299.320


## Metrics

In [ ]:
#| export
from sklearn.metrics import mean_squared_error

In [ ]:
#| export
class MaskedMetric:
    def __init__(self, metric): store_attr()
    def __call__(self, item, pred):
        row_sel, col_sel = ~item.mask.all(1), ~item.mask.all(0)
        data, mask, pred = item.data.to_numpy(), item.mask.to_numpy(), pred.to_numpy()
        assert not mask[:, col_sel][row_sel, :].any() # gap is a rectangle
        return self.metric(data[:, col_sel][row_sel, :], pred[:, col_sel][row_sel, :])   

In [ ]:
#| export
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values'))

In [ ]:
#| export
def normalize(x, mean, std): return (x - mean) / std 
class NormalizedMetric:
    def __init__(self, metric: MaskedMetric, mean, std): store_attr()
    def __call__(self, item, pred):
        item.data = normalize(item.data, self.mean, self.std)
        pred = normalize(pred, self.mean, self.std)
        return self.metric(item, pred)

In [ ]:
#| export
rmse_mask = MaskedMetric(rmse)

In [ ]:
rmse_mask(targ, pred)

array([0.27340029])

## Filling Comparison

Take a variable to be filled, makes an artificial gap of given len, tries to fill with 3 methods and return metrics for each of them, repeat `n_rep` times

In [ ]:
#| export
import random
import polars as pl
from tqdm.auto import tqdm

In [ ]:
#| export
class FillComparison():
    def __init__(self, model, df, control, block_len):
        store_attr()
        self.k_imp = KalmanImputation(model)
        self.era_imp = ERAImputation()
        self.mds_imp = MDSImputation("", df)
        self.methods = [self.k_imp, self.era_imp, self.mds_imp]
        self.rmse_norm = NormalizedMetric(MaskedMetric(rmse), *get_stats(df))
        
    def _imp_compare_single(self, gap_len, var, n_rep):
        """Compares `n_rep` times the imputation methods, for gap in `var` with len `gap_len`"""
        dls = imp_dataloader(hai, hai_era, var_sel = var, gap_len=10, block_len=100, control_lags = [1], n_rep=1, bs=1).cpu()
        items = random.choices(dls.items, k=n_rep)
        preds_k, targs = self.k_imp.preds_all(items = items, dls=dls)
        
        self.mds_imp.var = var
        
        out = []
        for targ, pred_k in zip(targs, preds_k):
            for imp in self.methods:
                pred = imp(targ) if imp is not self.k_imp else pred_k.mean #workaround for pipeline
                out.append(
                {'method': imp.name,
                 'rmse': rmse_mask(targ, pred).item(),
                 'rmse_norm': self.rmse_norm(targ, pred).item(),
                 'var': var,
                 'gap_len': gap_len
                }
                )
        return pl.DataFrame(out)
    
    def imp_compare(self, gap_len, var, n_rep):
        """Compare imputation performance for all combination of parameters"""
        arg_sets = list(product_dict(gap_len=tuplify(gap_len), var=tuplify(var)))
        out = []
        for arg_set in tqdm(arg_sets):
            out.append(self._imp_compare_single(**arg_set, n_rep=n_rep))
        return pl.concat(out)
        

In [ ]:
comp = FillComparison(model = model, df = hai, control = hai_era, block_len = 100)

In [ ]:
data_results = comp.imp_compare(gap_len = [5,10,20, 45], var=list(hai.columns), n_rep=10) 

  0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
# hai.corr()

## Plotting

In [ ]:
#| export
import altair as alt

In [ ]:
#| export
def facet_wrap(data: pd.DataFrame, # full dataset
               plot_fn,# function that makes the plot, takes 2 arguments: data and y_label
               col: str, # column to facet
               y_labels: list[str]|None =None, # custom labels y axis 
               n_cols=3
              ):
    col_vals = data[col].unique()
    plot_list = [alt.hconcat() for _ in range(0, len(col_vals), n_cols)]
    for i, col_v in enumerate(col_vals):
        plot = plot_fn(data[data[col]==col_v],
                       y_labels[i] if y_labels is not None else col_v
                      ).properties(title=col_v)
        plot_list[i // n_cols] |= plot
    return alt.vconcat(*plot_list).resolve_scale(
        y='independent'
    )
    

### The Plot 

In [ ]:
#| export
def _the_plot(data, y_label=rmse):
    return alt.Chart(data).mark_boxplot().encode(
        x = 'gap_len:N',
        y = alt.Y('rmse', title=y_label, scale=alt.Scale(domainMin=-.2 *data['rmse'].mean())),
        color=alt.Color('method:N', scale=alt.Scale(domain=["KalmanFilter", "ERA", "MDS"], scheme='dark2')),
        xOffset=alt.XOffset('method', scale=alt.Scale(domain=["KalmanFilter", "ERA", "MDS"])),
        # column=alt.Column('var')
    ).properties(width=200, height=180)

In [ ]:
#| export
def the_plot(data):
     return facet_wrap(data.to_pandas(), _the_plot, "var", y_labels = [f"rmse {var} [{units_big[var]}]" for var in data['var'].unique()])

In [ ]:
the_plot(data_results)

alt.VConcatChart(...)

## Gap Length

In [ ]:
# 18,20,25,30,40,50,60,70
gap_len = FillComparison(model, hai, hai_era, block_len=200).imp_compare(gap_len = [2,3,4,5,6,7,8,9,10,12,15,], var=['TA', 'SW_IN', 'VPD'], n_rep=10)

  0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
gap_len_agg = gap_len.groupby(['var', 'gap_len', 'method']).agg([
    pl.col("rmse").median().alias("median"),
    pl.col("rmse").quantile(.25).alias("Q1"),
    pl.col("rmse").quantile(.75).alias("Q3"),
    pl.col("rmse").min().alias("min"),
    pl.col("rmse").max().alias("max"),
    
]).to_pandas()
# ).with_column(
#     (pl.col("Q3") - pl.col("Q1")).alias("IRQ")
# ).with_columns([
#     (pl.col("Q1") - 1.5 * pl.col("IRQ")).alias("min"),
#     (pl.col("Q3") + 1.5 * pl.col("IRQ")).alias("max"),
# ])

In [ ]:
def facet_grid(data: pd.DataFrame, # full dataset
               plot_fn,# function that makes the plot, takes 2 arguments: data and y_label
               col: str, # column to facet,
               row: str,
               y_labels: list[str]|None =None, # custom labels y axis
              ):
    col_vals = data[col].unique()
    plots = []
    for i, col_v in enumerate(col_vals):
        plot = facet_wrap(data[data[col]==col_v], plot_fn, row, y_labels, n_cols=1)  
        plots.append(plot)
    return alt.hconcat(*plots)    

In [ ]:
def plot_gap_len(data, y_label):
    median = alt.Chart(data).mark_line().encode(
        x = alt.X('gap_len', title="gap_len"),
        y = alt.Y('median', title=y_label),
        color='var'
    ).properties(height=150, width=200)
    Qs = alt.Chart(data).mark_errorband().encode(x = 'gap_len', y = alt.Y('Q1', title=y_label), y2= 'Q3', color='var')
    # min = alt.Chart(gap_len_agg).mark_point().encode(x = 'gap_len', y = 'min', color='var')
    # max = alt.Chart(gap_len_agg).mark_point().encode(x = 'gap_len', y = 'max', color='var')

    return (median + Qs ) #+ min + max)

In [ ]:
plot_gap_len(gap_len_agg[gap_len_agg['var'] == 'TA'][gap_len_agg['method'] == 'ERA'], "label")

/tmp/ipykernel_213632/626086821.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  plot_gap_len(gap_len_agg[gap_len_agg['var'] == 'TA'][gap_len_agg['method'] == 'ERA'], "label")


alt.LayerChart(...)

In [ ]:
plot_gap_len(gap_len_agg, y_label="").facet(column='method', row='var').resolve_scale(y='independent')

alt.FacetChart(...)

In [ ]:
facet_wrap(gap_len_agg[gap_len_agg['var'] == 'TA'], plot_gap_len, 'method')

alt.VConcatChart(...)

In [ ]:
facet_grid(gap_len_agg, plot_gap_len, row='var', col='method')

alt.HConcatChart(...)

## Control

In [ ]:
class KalmanFilterSettingComparison():
    def __init__(self, model, df, control, block_len):
        store_attr()
        self.columns = list(df.columns)
        self.k_imp = KalmanImputation(model, df, control)
        
    def _imp_compare_single(self, gap_len, var, n_rep, gap_single_var, use_control):
        """Compares `n_rep` times the imputation methods, for gap in `var` with len `gap_len`"""
        self.model.use_control = use_control
        pipe, block_ids = imp_pipeline(self.df, self.control,
                                       [var] if gap_single_var else self.columns,
                                       gap_len=gap_len,
                                       block_len=self.block_len,
                                       control_lags = [1], n_rep=1)
        pipe = Pipeline(pipe[:-3]) 
        raw_items = random.choices(block_ids, k=n_rep)
    
        out = []
        for r_item in raw_items:
            item = pipe(r_item)
            pred = self.k_imp(item)
            out.append({
                 'rmse': rmse_mask(item, pred).item(),
                 'var': var,
                 'gap_len': gap_len,
                 'gap_single_var': gap_single_var,
                 'use_control': use_control
            })
        return pl.DataFrame(out)
    
    def imp_compare(self, gap_len, var, gap_single_var, use_control, n_rep):
        """Compare imputation performance for all combination of parameters"""
        args = {name: tuplify(val) for val, name in zip([gap_len, var, gap_single_var, use_control], ['gap_len', 'var', 'gap_single_var', 'use_control'])}
        arg_sets = list(product_dict(**args))
        out = []
        for arg_set in tqdm(arg_sets):
            out.append(self._imp_compare_single(**arg_set, n_rep=n_rep))
        return pl.concat(out)
        

In [ ]:
k_comp = KalmanFilterSettingComparison(model, hai, hai_era, block_len=100)

TypeError: KalmanImputation.__init__() takes 2 positional arguments but 4 were given

In [ ]:
use_control = k_comp.imp_compare(gap_len=10, var='TA', gap_single_var=True, use_control=[True, False], n_rep=30)

In [ ]:
def plot_control(data, y_label=rmse):
    return alt.Chart(data).mark_boxplot().encode(
        x = 'use_control',
        y = alt.Y('rmse', title=y_label),
        color=alt.Color('use_control')
    )

## Table

A table where in the rows there is 

In [ ]:
#| export
def the_table(data):
    results_table = data.groupby(['gap_len', 'method', 'var']).agg([
    pl.col("rmse").mean().alias("mean_rmse"),
    (pl.col("rmse").std() / pl.col("rmse").len()).alias("se_rmse"),
    pl.col("rmse").std().alias("std")
    ]).sort(['var', 'gap_len', 'method'])

    out = {}
    for var in result_table['var'].unique():
        out[var] = result_table.filter(pl.col('var') == var).select(["method", "gap_len", "mean_rmse", "std"]).to_pandas()
    display_as_row(out)
    

In [ ]:
the_table(data_results)

method,gap_len,mean_rmse,std
ERA,5,113.0194,203.5983
KalmanFilter,5,89.3215,50.0099
MDS,5,131.1921,217.6335
ERA,10,133.4173,186.8481
KalmanFilter,10,101.6301,44.1854
MDS,10,141.2409,169.1160
ERA,20,57.2332,75.7305
KalmanFilter,20,124.8296,107.5873
MDS,20,75.2629,98.2012
ERA,45,157.7204,171.2499
